In [4]:
from openai import OpenAI
client = OpenAI()
import json
import pandas as pd

In [61]:

batch_input_file = client.files.create(
  file=open("commands/batchinput.jsonl", "rb"),
  purpose="batch"
)

In [62]:
batch_input_file_id = batch_input_file.id

nesto = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

batch_input_file_id

'file-4zkl7lVGXUzZ5gvYJnXXpfRa'

In [63]:
nesto

Batch(id='batch_Z9hM4ER1TLmPWM0fXkWSZUBr', completion_window='24h', created_at=1724315557, endpoint='/v1/chat/completions', input_file_id='file-4zkl7lVGXUzZ5gvYJnXXpfRa', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1724401957, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [64]:
nesto.id

'batch_Z9hM4ER1TLmPWM0fXkWSZUBr'

In [65]:
nesto.output_file_id

In [66]:
batch_job = client.batches.retrieve(nesto.id)


In [75]:
batch_job.status

'completed'

In [67]:
batch_job.output_file_id

'file-gbr6B7dAJfsIkQdC0EQWKn0X'

In [52]:
file_list = client.files.list()
file_list

SyncPage[FileObject](data=[FileObject(id='file-hB8l332rrbox2I7UwC64slXS', bytes=638, created_at=1724314004, filename='batch_Dil24G7j9Tfx5kYSHsAgoTTu_error.jsonl', object='file', purpose='batch_output', status='processed', status_details=None), FileObject(id='file-4boNepsCWA7tR0EZcAlu8ZV9', bytes=604, created_at=1724313999, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-LdeD1rqnl5NKinD6ruB2OKwB', bytes=1179, created_at=1724313204, filename='batch_moIvyRlt4y7RPuXB5jbf1dc9_output.jsonl', object='file', purpose='batch_output', status='processed', status_details=None), FileObject(id='file-9hk6bYhnNaqjFNU9CGiRvEZJ', bytes=516, created_at=1724313194, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-xYbTaC9zM8gQLB6gzXRP6HqB', bytes=1192, created_at=1724312260, filename='batch_5wV532FUKDGnWqgFdhnkjZPS_output.jsonl', object='file', purpose='batch_o

In [53]:
file_list.data

[FileObject(id='file-hB8l332rrbox2I7UwC64slXS', bytes=638, created_at=1724314004, filename='batch_Dil24G7j9Tfx5kYSHsAgoTTu_error.jsonl', object='file', purpose='batch_output', status='processed', status_details=None),
 FileObject(id='file-4boNepsCWA7tR0EZcAlu8ZV9', bytes=604, created_at=1724313999, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None),
 FileObject(id='file-LdeD1rqnl5NKinD6ruB2OKwB', bytes=1179, created_at=1724313204, filename='batch_moIvyRlt4y7RPuXB5jbf1dc9_output.jsonl', object='file', purpose='batch_output', status='processed', status_details=None),
 FileObject(id='file-9hk6bYhnNaqjFNU9CGiRvEZJ', bytes=516, created_at=1724313194, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None),
 FileObject(id='file-xYbTaC9zM8gQLB6gzXRP6HqB', bytes=1192, created_at=1724312260, filename='batch_5wV532FUKDGnWqgFdhnkjZPS_output.jsonl', object='file', purpose='batch_output', status='proces

In [54]:
for file_info in file_list.data:
    if nesto.id in file_info.filename:
        print(file_info.id)
    # if file_info["filename"] == "your_file_name.txt":
    #     file_id = file_info["id"]
    #     print(f"File ID: {file_id}")
    #     break

file-hB8l332rrbox2I7UwC64slXS


In [68]:
file_response = client.files.content(batch_job.output_file_id)
print(file_response.text)
print(len(file_response.text.split("\n")))
for line in file_response.text.split("\n"):
    if line:
        obj = json.loads(line)
        print(obj)

{"id": "batch_req_up5WRFQzNWqInYkmol6LaLkj", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "fc20b91593553b798db59c48075bd41e", "body": {"id": "chatcmpl-9yxXDPpBTpPmuiK1fq8MTEBpdw84o", "object": "chat.completion", "created": 1724315559, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Hello! How can I assist you today?", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 20, "completion_tokens": 9, "total_tokens": 29}, "system_fingerprint": null}}, "error": null}
{"id": "batch_req_zxEKWQeVXM7plhCrIEDCqOiI", "custom_id": "request-2", "response": {"status_code": 200, "request_id": "2268077e22d32426f9259abd54efef51", "body": {"id": "chatcmpl-9yxXEyTkiGoDCutnKi7zEgHxbQB8z", "object": "chat.completion", "created": 1724315560, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Greetings. How may I not assist you today?", "refus

In [70]:
res = client.files.content(batch_job.output_file_id).content

In [71]:
result_file_name = "commands/batch_job_results_movies.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(res)

In [72]:
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [73]:
results

[{'id': 'batch_req_up5WRFQzNWqInYkmol6LaLkj',
  'custom_id': 'request-1',
  'response': {'status_code': 200,
   'request_id': 'fc20b91593553b798db59c48075bd41e',
   'body': {'id': 'chatcmpl-9yxXDPpBTpPmuiK1fq8MTEBpdw84o',
    'object': 'chat.completion',
    'created': 1724315559,
    'model': 'gpt-3.5-turbo-0125',
    'choices': [{'index': 0,
      'message': {'role': 'assistant',
       'content': 'Hello! How can I assist you today?',
       'refusal': None},
      'logprobs': None,
      'finish_reason': 'stop'}],
    'usage': {'prompt_tokens': 20, 'completion_tokens': 9, 'total_tokens': 29},
    'system_fingerprint': None}},
  'error': None},
 {'id': 'batch_req_zxEKWQeVXM7plhCrIEDCqOiI',
  'custom_id': 'request-2',
  'response': {'status_code': 200,
   'request_id': '2268077e22d32426f9259abd54efef51',
   'body': {'id': 'chatcmpl-9yxXEyTkiGoDCutnKi7zEgHxbQB8z',
    'object': 'chat.completion',
    'created': 1724315560,
    'model': 'gpt-3.5-turbo-0125',
    'choices': [{'index': 0,

In [83]:
from pathlib import Path
from typing import List, Dict


def save_failed_ids(failed: List[Dict[str, str]], dataset_name: str) -> None:
    file_path = Path(f"failed/failed_{dataset_name}.json")
    file_path.parent.mkdir(parents=True, exist_ok=True)
    # Write the IDs to a text file, one per line
    with open(file_path, "w") as f:
        # Convert exceptions to string because exceptions are not JSON serializable
        json.dump(failed, f, default=str, indent=4)

def make_dataset(
    processed_commands_path: Path, dataset_name: str
) -> Dict[str, str | List[str]]:
    returned_dict = {
        "id": [],
        "query": [],
        "passage_text": [],
    }
    failed = []
    # Open and iterate through the .jsonl file
    with open(processed_commands_path, "r") as file:
        for line in file:
            id_ = None
            print(line)
            try:
                data = json.loads(line)
                id_ = data["custom_id"].replace("task-", "")
                returned_data = data['response']['body']['choices'][0]['message']['content']
                tranlation = json.loads(
                    returned_data
                )  # gpt message i.e. translation in this case
                returned_dict["id"].append(id_)
                returned_dict["query"].append(tranlation["query"])
                returned_dict["passage_text"].append(tranlation["passage_text"])
            except Exception as e:
                failed.append({"id": id_, "exception": e})
    if failed:
        save_failed_ids(failed, dataset_name=dataset_name)
    return returned_dict

In [84]:
result_file_name = "commands/results_msmarco.jsonl"

a = make_dataset(processed_commands_path=result_file_name, dataset_name='msmarco')
a

{"id": "batch_req_st8Ee5AZ9Z8IouuXaa6O9ZzB", "custom_id": "task-0", "response": {"status_code": 200, "request_id": "bc8e8afe52b6f73bd609691b47eefd99", "body": {"id": "chatcmpl-9yy7fLf2PpHJxAg7Y30xUps2MbClh", "object": "chat.completion", "created": 1724317819, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n    \"query\": \"da li ljudska kosa zaustavlja veverice\",\n    \"passage_text\": [\"Hranimo veverice u na\u0161em dvori\u0161tu tokom jeseni i zime i primetili smo da neke od njih imaju \u0107elave fleke. Jedna ima nedostatak krzna du\u017e le\u0111a i ispod oba pazuha. Tako\u0111e, druga ima nekoliko mesta bez krzna na celom grudnom ko\u0161u. Sve one jedu i \u010dini se da imaju dobar apetit.\", \"\u017divotinje ne podnose miris ljudske kose, pa posipanje barijere od odse\u010denih dlaka oko va\u0161e ba\u0161te, ili lagano udubljivanje u zemlju prilikom sadnje lukovica, o\u010digledno ima neku vrednost. \u010citava situacija

{'id': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'query': ['da li ljudska kosa zaustavlja veverice',
  'Koji su benefiti fosilnih goriva?',
  'šta je apotel',
  'prosečna cena za prilagođeni baldahin',
  'šta je hardver na računaru',
  'definicija logsitike EDI',
  'zašto bi trebalo da bude ilegalno rekreativno korišćenje marihuane',
  'U koju klasu spadaju paukovi?',
  'gde se nalazi amurski leopard',
  'koliko košta da se sterilizuje moja ženka kunića'],
 'passage_text': [['Hranimo veverice u našem dvorištu tokom jeseni i zime i primetili smo da neke od njih imaju ćelave fleke. Jedna ima nedostatak krzna duž leđa i ispod oba pazuha. Takođe, druga ima nekoliko mesta bez krzna na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.',
   'Životinje ne podnose miris ljudske kose, pa posipanje barijere od odsečenih dlaka oko vaše bašte, ili lagano udubljivanje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava situacija me pomalo nasmejava. Nika

## Checking tanslation results

In [47]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

msmarco_path = "datasets/msmarco.parquet"
pd.read_parquet(msmarco_path, engine='fastparquet')

,id,query,passage_text
0,1,Koji su benefiti fosilnih goriva?,[Najveća prednost korišćenja fosilnih goriva je ta što se lako mogu skladištiti i transportovati sa jednog mesta na drugo. Velike rezerve uglja se stoga prenose iz rudnika uglja do industrijskih postrojenja koja su udaljena od rudnika. Nafta se takođe prenosi do udaljenih elektrana kako bi se pr...
1,2,šta je apotema,"[Apotema. Apotema pravilnog mnogougla je segment nacrtan od centra mnogougla (tj. centra kruga koji opisuje mnogougao) do jedne strane, tako da je normalan na stranu. Ako pravilni mnogougao ima površinu od A kvadratnih jedinica, obim od P jedinica, apotemu od a jedinica, tada je površina jednaka..."
2,3,average cost for custom canopy,"[Čestitamo! Pronašli ste BuyShade.com, vaš izvor za šatore koji se brzo postavljaju, prekrivače za stolove, prilagođene šatore, štampane zastave i još mnogo toga. Sa stotinama konfiguracija proizvoda i stilova u našem portfoliju i timom vrhunskih dizajnera, specijalizovani smo za pravljenje pril..."
3,4,šta je hardver u računaru,"[Hardver se najbolje opisuje kao uređaj, poput tvrdog diska, koji je fizički povezan sa računarom ili nešto što se može fizički dodirnuti. CD-ROM, monitor računarskog ekrana, štampač i grafička kartica su primeri računarskog hardvera. Hardver služi kao sistem za isporuku softverskih rešenja. Har..."
4,5,edi logistics definition,"[Kao i mnoge druge rane informacione tehnologije, EDI je bio inspirisan razvojem vojne logistike. Složenost berlinske blokade iz 1948. zahtevala je razvoj koncepata i metoda za razmenu, ponekad preko 300 baud teletipskog modema, ogromnih količina podataka i informacija o prevoženim robama. U EDI..."
5,6,Zašto bi trebalo da bude ilegalno rekreativno korišćenje marihuane?,"[Neki se pitaju 'zašto bi marihuana trebalo da bude legalizovana?' ali mi bismo trebali da se zapitamo Zašto bi marihuana trebalo da bude ilegalna?. Sa filozofskog stanovišta, pojedinci zaslužuju pravo da sami donose odluke. Vlada ima pravo da ograniči te odluke samo ako postupci pojedinca ugrož..."
6,7,U koju klasu spadaju paukovi?,"[PODFAKULTET HELICERATA, KLASA ARACHNIDA. Paukovi. Ova grupa sadrži mnoge poznate organizme, uključujući paukove, grinje, škorpije i krpelje. Pogledajte velikog pauka sa desne strane. Ponovo primetite da postoje dva dela tela, glavotučak i abdomen. Na glavotučku se nalaze dva do četiri para jedn..."
7,9,Koliko košta da sterilizujem svoju ženku kunića?,"[Rizik od reproduktivnih karcinoma (jajnika, materice, mlečnih žlezda) kod nekastrirane ženke kunića gotovo je eliminisan sterilizacijom vaše ženke kunića. Vaš kastrirani mužjak kunić takođe će živeti duže, s obzirom na to da neće biti izazvan da se bori sa drugim životinjama (kunićima, mačkama,..."


In [46]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

nq_path = "datasets/naquestions.parquet"
pd.read_parquet(nq_path, engine='fastparquet')

,id,query,passage_text
0,-5501481664893105662,ko je predložio da elektroni pokazuju svojstva talasa i čestica,"[Kroz rad Maks Planka, Alberta Einštajna, Luj de Broglia, Artura Komptona, Nilsa Bora i mnogih drugih, trenutna naučna teorija tvrdi da sve čestice takođe imaju prirodu talasa (i obrnuto). Ovaj fenomen je potvrđen ne samo za elementarne čestice, već i za složene čestice poput atoma pa čak i mole..."
1,8594030243394572667,Koliko senatora ima u Senatu SAD-a?,"[Senat Sjedinjenih Američkih Država se sastoji od 100 članova, po dva iz svake od 50 saveznih država. U nastavku je lista trenutnih senatora SAD-a koji sede u 115. Kongresu Sjedinjenih Američkih Država.]"
2,-716176363900512091,Faza promene gasa u čvrsto se naziva,"[Depozicija je termodinamički proces, faza prelaska u kojoj gas prelazi u čvrsto stanje bez prolaska kroz tečnu fazu. Suprotno od depozicije je sublimacija i zbog toga se ponekad depozicija naziva desublimacija.]"
3,-5593761806871287545,Koja je najduža engleska reč u rečniku?,"[Najduža reč u bilo kom od glavnih engleskih jezičkih rečnika je pneumonoultramicroscopicsilicovolcanoconiosis, reč koja se odnosi na bolest pluća koja se dobija udisanjem veoma finih čestica silicijuma, specifično sa vulkana; medicinski, isto značenje ima kao silikoza. Reč je namerno skovana ka..."
4,2074579308847994444,koliki procenat Zemljine površine je voda,"[Voda je raspoređena širom Zemlje. Većina vode u Zemljinoj atmosferi i kori potiče iz slane morske vode svetskog okeana, dok slatka voda čini samo 2.5% ukupne količine. Zbog okeana koji pokrivaju otprilike 78% površine Zemlje reflektujući plavu svetlost, Zemlja izgleda plavo iz svemira i često s..."
5,-4418682909598136345,Odakle potiče supa od gnezda ptica?,"[Jesti gnezda ptica su gnezda ptica koja stvaraju ptice lastavice koristeći zgrušanu pljuvačku, koja se bere radi ljudske konzumacije. Posebno su cenjena u kineskoj kulturi zbog svoje retkosti, navodno visoke nutritivne vrednosti i izuzetnog ukusa. Jesti gnezda ptica su među najskupljim životinj..."
6,-4809486218828226485,Odakle dolaze neutroni u nuklearnoj fisiji?,"[Izvor neutrona za pokretanje je izvor neutrona koji se koristi za stabilno i pouzdano pokretanje lančane nuklearne reakcije u nuklearnim reaktorima, kada su punjeni svežim nuklearnim gorivom, čiji neutronski fluks od spontane fisije nije dovoljan za pouzdano pokretanje, ili nakon produženih per..."


In [74]:
from openai import OpenAI
client = OpenAI()
# nq
client.batches.retrieve("batch_npW3rHFahgV8w5PIQv2hUlGl")

Batch(id='batch_npW3rHFahgV8w5PIQv2hUlGl', completion_window='24h', created_at=1724339557, endpoint='/v1/chat/completions', input_file_id='file-0JYiyJR85pe4GLUysU69BSbw', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1724339630, error_file_id=None, errors=None, expired_at=None, expires_at=1724425957, failed_at=None, finalizing_at=1724339629, in_progress_at=1724339557, metadata=None, output_file_id='file-gIDcTrku4M9rzUjZpXzNzaUM', request_counts=BatchRequestCounts(completed=10, failed=0, total=10))

In [54]:
from openai import OpenAI
client = OpenAI()
# ms_marco
client.batches.retrieve("batch_qFbTrX90NiVLCZ9ij4WtOljJ")

Batch(id='batch_qFbTrX90NiVLCZ9ij4WtOljJ', completion_window='24h', created_at=1724333601, endpoint='/v1/chat/completions', input_file_id='file-L8mLrI8xszNDhmjFVBOtn8X5', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1724333675, error_file_id=None, errors=None, expired_at=None, expires_at=1724420001, failed_at=None, finalizing_at=1724333674, in_progress_at=1724333601, metadata=None, output_file_id='file-QhzI3UzzXhcSPcunIY0hdBCb', request_counts=BatchRequestCounts(completed=10, failed=0, total=10))

In [70]:
import json
from pprint import pprint

# Define the path to the .jsonl file
file_path = 'commands/results_msmarco.jsonl'

# Open and load the .jsonl file
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

# Now 'data' contains a list of dictionaries, each representing a line in the .jsonl file
pprint(data)


[{'custom_id': 'task-0',
  'error': None,
  'id': 'batch_req_sPXUQ5natcpebt9UqSWfyVY0',
  'response': {'body': {'choices': [{'finish_reason': 'stop',
                                     'index': 0,
                                     'logprobs': None,
                                     'message': {'content': '{\n'
                                                            '    "query": "Da '
                                                            'li ljudska kosa '
                                                            'zaustavlja '
                                                            'veverice?",\n'
                                                            '    '
                                                            '"passage_text": '
                                                            '[\n'
                                                            '        "Hrani '
                                                            'smo naše veverice '


In [1]:
output = {'id': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'query': ['Da li ljudska kosa zaustavlja veverice?', 'what is a apothem', 'Šta je hardver u računaru?', 'why should recreational marijuana be illegal', 'U kojoj klasi su paukovi?', 'where is the amur leopard found'], 'passage_text': [['Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima nedostatak dlake. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nedostatak dlake na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.', 'Životinje ne podnose miris ljudske kose, pa posipanje barijere od odsečene kose oko vaše bašte, ili lagano uklapanje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako natera da se nasmejem. Nikada mi nije palo na pamet da smo mi ti koji smrde.', 'Rasporedite malo ljudske kose oko vaših bašti sa povrćem i cvećem. To će uplašiti veverice jer su ljudi predator veverica. Bolje je ako kosa nije oprana tako da će veverice lako uhvatiti ljudski miris.', '1 Takođe možete posuti krvavu brašnu oko vaše bašte. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veverica izuzetno visoka. 4 Takođe, ako je životinja ženka, postoji velika verovatnoća da ćete je odvojiti od beba koje zavise od nje za preživljavanje.', 'Gubitak dlake kod veverica može biti uzrokovan šugaravicom ili gljivičnom bolešću. Šugaravica je bolest izazvana mikroskopskim grinjama koje se uvlače ispod kože. Grinja šugaravice kod veverica je zabeležena kod lisica i sivih veverica. Nema izveštaja da je ovaj tip šugaravice prenet na ljude ili kućne ljubimce.', '1 Takođe možete rasuti pseću ili ljudsku kosu oko vaše bašte. 2 Jedan čitalac deli, "Imao sam problem sa vevericama koje su kopale moje lukovice. 3 Sada, jednom u proleće i jednom u jesen, tražim od svog frizera da mi sačuva veliku kesu kose. 4 Lagano je ukopavam u zemlju.', 'Sredstvo za odbijanje zečeva. Ljudska kosa će držati zečeve van vaše bašte!! Skupite kosu sa četki i rasporedite je oko vaše bašte! (preko Pioneer Thinking). Prirodna malč. Kada se isplete u prostirku, zadržava vlagu, odvraća korov. U nekim slučajevima, zapravo može smanjiti eroziju zemljišta. (preko NPR). Biljni đubrivo.'], ['Apothem. Apotema pravilnog mnogougla je segment nacrtan od centra mnogougla (tj. centra kruga koji opisuje mnogougao) do jedne strane, tako da je normalan na stranu. Ako pravilni mnogougao ima površinu od A kvadratnih jedinica, obim od P jedinica, apotemu od a jedinica, tada je površina jednaka polovini proizvoda obima i apoteme. Ipak, budite oprezni sa ovom formulom: za datu apotemu, postoji SAMO JEDNA MOGUĆA vrednost za P.', 'Apoteme. Apotema pravilnog mnogougla je rastojanje od centra do središta jedne od njegovih strana. Apotema kvadrata je jednaka polovini dužine jedne strane.', "Apotema pravilnog mnogougla. Definicija: Linijski segment od centra pravilnog mnogougla do središta strane. Pokušajte ovo Prilagodite donji mnogougao povlačenjem bilo koje narandžaste tačke ili promenom broja strana. Obratite pažnju na ponašanje plave linije apoteme. Reč apotema može se odnositi na samu liniju ili dužinu te linije. Dakle, možete tačno reći 'nacrtajte apotemu' i 'apotema je 4cm'. Svaka formula ispod pokazuje kako pronaći dužinu apoteme pravilnog mnogougla. Koristite formulu koja koristi činjenice koje su vam date za početak.", 'više ... Rastojanje od centra pravilnog mnogougla do središta jedne strane. (Za krug je to rastojanje od centra do središta tetive.). Pravilni mnogougli - osobine.', "Apotema je takođe poluprečnik inkruzne kružnice mnogougla. Za mnogougao sa n strana, postoji n mogućih apotema, sve iste dužine naravno. Reč apotema može se odnositi na samu liniju ili dužinu te linije. Dakle, možete tačno reći 'nacrtajte apotemu' i 'apotema je 4cm'. Svaka formula ispod pokazuje kako pronaći dužinu apoteme pravilnog mnogougla. Koristite formulu koja koristi činjenice koje su vam date za početak.", 'Apotema šestougla. Apotema pravilnog mnogougla je linija od centra do središta jedne od njegovih strana. Ekvivalentno, to je linija nacrtana od centra mnogougla koja je normalna na jednu od njegovih strana. Reč apotema takođe može se odnositi na dužinu tog linije. Zbog toga će sve apoteme u mnogouglu biti kongruentne. Za pravilnu piramidu, koja je piramida čija je osnovica pravilan mnogougao, apotema je kosina visina bočne strane; to jest, najkraće rastojanje od vrha do osnove na datoj strani.', 'Za jednakostranični trougao, apotema je ekvivalentna liniji od središta strane do bilo kog od centara trougla, budući da se centri jednakostraničnog trougla poklapaju kao posledica definicije. Zbog toga će sve apoteme u mnogouglu biti kongruentne. Za pravilnu piramidu, koja je piramida čija je osnovica pravilan mnogougao, apotema je kosina visina bočne strane; to jest, najkraće rastojanje od vrha do osnove na datoj strani.', 'Za dat krug, apotema je normalno rastojanje od središta tetive do centra kruga. Takođe je jednaka poluprečniku minus sagita. Za pravilan mnogougao, apotema jednostavno predstavlja rastojanje od centra do strane, tj. inpoluprečnik mnogougla. POGLEDAJTE TAKOĐE: Tetiva, Sektor kruga, Segment kruga, Inpoluprečnik, Prečnik, Pravilan mnogougao, Sagita CITIRAJTE OVO KAO: Weisstein, Eric W. Apotema.. Iz MathWorld --A Wolfram Web Resursa.', 'Apotema je linija od centra pravilnog mnogougla do središta jedne od njegovih strana. Takođe je definisana kao poluprečnik inkruzne kružnice mnogougla. Za pravilan mnogougao sa n strana, postoji n mogućih apotema iste dužine. Ako pretpostavimo pravougli trougao gde je apotema priležna stranica, polovina strane mnogougla suprotna stranica i linija koja spaja centar mnogougla i vrh mnogougla hipotenuza. Pretpostavimo da je ugao formiran priležnom stranicom i hipotenuzom $\\theta$.'], ['Hardver se najbolje opisuje kao uređaj, poput tvrdog diska, koji je fizički povezan sa računarom ili nešto što se može fizički dodirnuti. CD-ROM, monitor računarskog ekrana, štampač i grafička kartica su primeri računarskog hardvera. Hardver služi kao sistem za isporuku softverskih rešenja. Hardver računara se retko menja, u poređenju sa softverom i podacima, koji su "meki" u smislu da se lako stvaraju, menjaju ili brišu na računaru. Da biste obavili određeni zadatak, potrebno je da ga završite.', 'Računarski hardver (obično jednostavno nazvan hardver kada je računarski kontekst podrazumevan) je skup fizičkih elemenata koji čine računarski sistem. Za razliku od toga, softver su instrukcije koje mogu biti smeštene i pokrenute od strane hardvera. Softver je bilo koji skup mašinski čitljivih instrukcija koje usmeravaju procesor računara da obavi određene operacije. Kombinacija hardvera i softvera čini upotrebljiv računarski sistem.', 'Računarski hardver su fizički delovi ili komponente računara, poput monitora, miša, tastature, skladištenja podataka na računaru, hard diska (HDD), grafičkih kartica, zvučnih kartica, memorije, matične ploče i tako dalje, sve fizičke objekte koji su opipljivi. Za razliku od toga, softver su instrukcije koje mogu biti smeštene i pokrenute od strane hardvera. Softver je bilo koji skup mašinski čitljivih instrukcija koje usmeravaju procesor računara da obavi određene operacije. Kombinacija hardvera i softvera čini upotrebljiv računarski sistem.', 'Računarski hardver je skup fizičkih delova računarskog sistema. To uključuje kućište računara, monitor, tastaturu i miša. Takođe uključuje sve delove unutar kućišta računara, poput hard diska, matične ploče, grafičke kartice i mnogih drugih. Računarski hardver je ono što možete fizički dodirnuti.', 'Definicija računarskog hardvera. Hardver je sveobuhvatan pojam za sve fizičke delove računara, za razliku od podataka koje sadrži ili na kojima radi, i softvera koji pruža instrukcije hardveru za obavljanje zadataka.', 'Skraćeno kao HW, hardver se najbolje opisuje kao biloja fizička komponenta računarskog sistema koja sadrži matičnu ploču, IC-ove ili druge elektronike. Savršen primer hardvera je ekran na kojem gledate ovu stranicu. Bez ikakvog hardvera, vaš računar ne bi postojao, a softver se ne bi mogao koristiti. Slika je Logitech web kamera, primer spoljnog hardverskog perifernog uređaja. Ovaj hardverski uređaj omogućava korisnicima da snimaju video zapise ili slike i prenose ih preko interneta.', 'Definicije. Računarski sistem se sastoji od dva glavna elementa: hardvera i softvera. Računarski hardver je skup svih delova koje možete fizički dodirnuti. Sa druge strane, softver nije nešto što možete dodirnuti. Softver je skup instrukcija za računar da obavi određene operacije. To uključuje kućište računara, monitor, tastaturu i miša. Takođe uključuje sve delove unutar kućišta računara, poput hard diska, matične ploče, grafičke kartice i mnogih drugih. Računarski hardver je ono što', 'Računarski hardver je bilo koji fizički uređaj koji se koristi u ili sa vašim računarom, dok je softver skup koda instaliran na hard disku vašeg računara. Na primer, monitor računara koji koristite za čitanje ovog teksta i miš koji koristite za navigaciju ove veb stranice su računarski hardver. Svi softveri koriste barem jedan hardverski uređaj za rad. Na primer, video igra, koja je softver, koristi procesor računara (CPU), memoriju (RAM), hard disk i grafičku karticu za rad. Softver za obradu teksta koristi procesor računara, memoriju i hard disk za kreiranje i čuvanje dokumenata.', 'Uvod. Računarski hardver je fizički deo računara, za razliku od onog koji se izvršava ili pokreće na hardveru. Hardver računara se retko menja, dok se softver i podaci često menjaju. Izraz "mek" se odnosi na lako stvaranje, modifikovanje ili brisanje', 'Računarski hardver se odnosi na fizičke komponente koje čine računarski sistem. Postoji mnogo različitih vrsta hardvera koji se mogu instalirati unutar, i povezati sa spoljašnje strane, računara.'], ["Neke osobe pitaju 'zašto bi marihuana trebalo da bude legalizovana?' ali mi bismo trebali pitati Zašto bi marihuana trebalo da bude ilegalna?. Sa filozofskog stanovišta, pojedinci zaslužuju pravo da sami donose odluke. Vlada ima pravo da ograniči te odluke samo ako postupci pojedinca ugrožavaju nekog drugog. REZIME: Imali bismo više novca za važne probleme da je marihuana legalna. Treći glavni razlog zašto bi marihuana trebala biti legalna je zato što prohibicija na bilo koji način ne pomaže zemlji, već uzrokuje mnogo problema.", 'Dok bi marihuanski lekovi trebali biti dostupni na recept iz apoteke, upotreba trave ne bi trebala biti normalizovana i marihuana bi trebala biti ilegalna. Bob Enyart je pastor Denver Bible crkve u Arvadi, Kolorado. Molimo dodajte korisničko ime da biste videli ili dodali komentare. Što se tiče upotrebe marihuane ili bilo kojeg moralnog pitanja, rasprava o ispravnom i pogrešnom u današnjem društvu je posebno teška jer sve veći broj ljudi odbacuje apsolutno ispravno i pogrešno.', 'Marihuana je najčešće zloupotrebljena ilegalna droga u SAD-u i širom sveta. Oni koji podržavaju njenu legalizaciju, bilo iz medicinskih ili opštih razloga, ne prepoznaju da najveći troškovi marihuane nisu povezani sa njenom prohibicijom; već su troškovi rezultat same upotrebe marihuane. Mnogi ljudi koji žele legalizaciju marihuane strastveni su oko svoje percepcije navodnih neuspeha politika usmerenih na smanjenje upotrebe marihuane, ali ti zagovornici legalizacije retko—ako ikada—opisuju svoj plan za oporezivanje i regulisanje marihuane kao legalne droge.', 'Troškovi legalizacije marihuane ne bi bili samo finansijski. Novi korisnici marihuane ne bi bili ograničeni na odrasle osobe ako bi marihuana bila legalizovana, baš kao što regulative za alkohol i duvan ne sprečavaju upotrebu od strane mladih. Mnogi ljudi koji žele legalizaciju marihuane strastveni su oko svoje percepcije navodnih neuspeha politika usmerenih na smanjenje upotrebe marihuane, ali ti zagovornici legalizacije retko—ako ikada—opisuju svoj plan za oporezivanje i regulisanje marihuane kao legalne droge.', 'Rekreativna upotreba marihuane samo će dovesti do većeg stepena zavisnosti od droga u Sjedinjenim Američkim Državama. Kada legalizujemo rekreativnu upotrebu marihuane, to je klizav teren ka legalizaciji drugih ilegalnih droga koje su jače. Zato što ljudi žele da je koriste rekreativno i to smanjuje rizik od hapšenja zbog upotrebe. Zato što korisnici marihuane manje verovatno štete nekome i takođe ima zdravstvene beneficije kao što leči rak i druge bolesti/bolesti poput dijabetesa i drugih štetnih bolesti. Alkohol je nekada bio ilegalan. Prijavi post.', 'Zašto ne bismo trebali legalizovati marihuanu. DELJENJA. Za razliku od verovanja onih koji zagovaraju legalizaciju marihuane, trenutne uravnotežene, restriktivne i bipartitne politike droga Sjedinjenih Američkih Država prilično dobro funkcionišu i doprinele su smanjenju stope upotrebe marihuane u našoj naciji. Mnogi ljudi koji žele legalizaciju marihuane strastveni su oko svoje percepcije navodnih neuspeha politika usmerenih na smanjenje upotrebe marihuane, ali ti zagovornici legalizacije retko—ako ikada—opisuju svoj plan za oporezivanje i regulisanje marihuane kao legalne droge.', 'Ako bi SAD legalizovale marihuanu, broj korisnika marihuane bi se povećao. Danas postoji 15,2 miliona trenutnih korisnika marihuane u poređenju sa 129 miliona korisnika alkohola i 70,9 miliona korisnika duvana. Mnogi ljudi koji žele legalizaciju marihuane strastveni su oko svoje percepcije navodnih neuspeha politika usmerenih na smanjenje upotrebe marihuane, ali ti zagovornici legalizacije retko—ako ikada—opisuju svoj plan za oporezivanje i regulisanje marihuane kao legalne droge.', 'Marihuana ne bi trebala biti legalizovana za rekreativnu upotrebu jer će imati drastične negativne efekte na američku ekonomiju. Osobe koje su pod uticajem narkotika poput marihuane nisu sposobne da funkcionišu na način koji im omogućava obavljanje svakodnevnih zadataka potrebnih za posao. Zato što ljudi žele da je koriste rekreativno i to smanjuje rizik od hapšenja zbog upotrebe. Zato što korisnici marihuane manje verovatno štete nekome i takođe ima zdravstvene beneficije kao što leči rak i druge bolesti/bolesti poput dijabetesa i drugih štetnih bolesti. Alkohol je nekada bio ilegalan. Prijavi post.', 'Po mom ličnom mišljenju, marihuana bi trebala biti legalizovana za rekreativnu upotrebu. Ne samo što je statistički najsigurnija od svih droga korišćenih u Americi ili širom sveta, već nema prijavljenih smrtnih slučajeva od upotrebe. Zato što ljudi žele da je koriste rekreativno i to smanjuje rizik od hapšenja zbog upotrebe. Zato što korisnici marihuane manje verovatno štete nekome i takođe ima zdravstvene beneficije kao što leči rak i druge bolesti/bolesti poput dijabetesa i drugih štetnih bolesti. Alkohol je nekada bio ilegalan. Prijavi post.'], ['PODFAKULTET CHELICERATA, KLASA ARACHNIDA. Paukovi. Ova grupa sadrži mnoge poznate organizme, uključujući paukove, grinje, škorpije i krpelje. Pogledajte velikog pauka sa desne strane. Ponovo primetite da postoje dva dela tela, glavotučak i abdomen. Na glavotučku se nalaze dva do četiri para jednostavnih očiju.', 'Klasa Arachnida obuhvata raznovrsnu grupu člankonožaca: paukove, škorpije, krpelje, grinje, kosarke i njihove rođake. Naučnici opisuju preko 75.000 vrsta arahnida, većina njih su paukovi. Većina arahnida su mesožderi, obično se hrane insektima, i terestrični, živeći na kopnu.', 'Paukovi pripadaju tipu Arthropoda, zajedno sa insektima i rakovima. Red paukova, Araneae - zajedno sa škorpijama, kosarkama i velikim redom grinja i krpelja - čine klasu Arachnida. Paukovi se razlikuju od drugih arahnida po tome što im je telo podeljeno na glavotučak i abdomen.', 'Klasa Araneae sadrži paukove, veliku i fascinantnu grupu. Paukovi imaju tipične karakteristike arahnida - osam nogu, pedipalpe, helicere i dva glavna dela tela. Međutim, helicere su izgubile unutrašnji palac, a spoljni zglobni segment je postao zub.', 'Čineći svet boljim, jedan odgovor odjednom. Klasa pauka je Arachnida. Paukovi pripadaju arahnidnoj porodici zajedno sa svim drugim vrstama paukova. Paukovi nisu insekti. Oni su arahnidi. Takođe su škorpije, grinje, krpelji, kosarke i mnogi drugi.', 'Prema tome, pauk je smešten u tip Arthropoda, klasu (classis) Arachnida, red (ordo) Araneae. Ovaj red je dalje podeljen na 3 podreda. Mygalomorphae (primitivni paukovi), Aranaeomorphae (moderni paukovi) i Mesothelae, sa jednom porodicom paukova Liphistiidae.'], ['Amurski leopardi imaju najseverniji opseg od svih podvrsta leoparda. Amurski leopardi se nalaze u istočnoj Rusiji, blizu reke Amur unutar uskog pojasa južno od jezera Khanka; i u provinciji Jilin u Kini.', 'RHWA je sponzor Tigris Fondacije. Tigris Fondacija podržava očuvanje amurskih leoparda. Za više informacija, posetite veb sajt Tigris Fondacije. Leopardi (Panthera pardus) su među najprilagodljivijim vrstama mačaka i nalaze se u različitim staništima širom Afrike i Azije.', 'Amurski leopard (Panthera pardus orientalis), takođe poznat kao mandžurski leopard, je divlji mačji predator koji je autohton u planinskim područjima tajge, kao i drugim umerenim šumama u Koreji, severoistočnoj Kini i ruskom Dalekom istoku.', 'Saznajte više o ovom članku. Amurski leopard (Panthera pardus orientalis) je podvrsta leoparda autohtona za Primorski region jugoistočne Rusije i provinciju Jilin na severoistoku Kine. Klasifikovan je kao Kritično ugrožena vrsta od 1996. godine od strane IUCN-a.', 'Amurski leopard (Panthera pardus orientalis) je podvrsta leoparda autohtona za Primorski region jugoistočne Rusije i provinciju Jilin na severoistoku Kine. Klasifikovan je kao Kritično ugrožena vrsta od 1996. godine od strane IUCN-a. Procenjeno je da je 2007. godine preživelo samo 19-26 divljih amurskih leoparda.', 'Većina ljudi zapravo nije svesna gde žive amurski leopardi! Amurski leopardi žive u Severnoj Koreji, Rusiji, Kini i Japanu. Činjenice o amurskim leopardima o njihovom staništu pokazuju da se ovi životinje nalaze na obali Japana i severnom basenu Suchan. Herman Šlegel bio je prvi istraživač koji je otkrio ove životinje još 1857. godine.']], 'explanation': ['The translation aims to accurately convey the information about using human hair to deter squirrels and other animals from gardens. The translation maintains the original context and provides a natural-sounding Serbian version.', "The translation aims to accurately convey the concept of 'apothem' in the context of regular polygons, emphasizing its geometric properties and relationships. The Serbian translation maintains the technical terminology while ensuring clarity and coherence.", 'The translation aims to accurately convey the meaning of computer hardware in Serbian, ensuring that the terminology is appropriate and understandable to the target audience.', 'The translation aims to accurately convey the arguments presented in the passages regarding the reasons why recreational marijuana should be illegal. The translation maintains the original context and tone of the text, providing a clear and coherent Serbian version.', 'The translation accurately conveys the information about spiders belonging to the class Arachnida, along with additional details about their characteristics and classification within the animal kingdom.', 'The translation aims to accurately convey the information about the habitat of Amur leopards, ensuring that the context and details are preserved in Serbian.']}

In [2]:
len(output['id'])

10

In [78]:
output.keys()

dict_keys(['id', 'query', 'passage_text', 'explanation'])

In [3]:
len(output['query'])

6

In [4]:
len(output['passage_text'])

6

In [5]:
len(output['explanation'])

6